In [79]:
#display_read_well_csv()

# def get_well_info(filename='elisawells.csv',
#                  rows=8):
#     import csv, sys
    
#     rows = rows
#     row_count = 0
#     col_count = 0
    
#     # create list of lists
#     col_list = list()
    
#     with open(filename, 'rb') as f:
#         reader = csv.reader(f)
#         try:
#             col_count_list = list()
            
#             for row in reader:
#                 row_value = int(row[0])
                
#                 if row_count >= rows:
#                     row_count = 0
#                     col_count = col_count + 1
#                     #print col_count_list
#                     col_list.append(col_count_list)
#                     col_count_list = list()
                
#                 row_count = row_count + 1
#                 col_count_list.append(row_value)
                
#                 #print '%i\t%i\t%i' % (row_count, col_count, row_value)
            
#             if col_count_list:
#                 col_list.append(col_count_list)
            
#         except csv.Error as e:
#             # TODO handle
#             print 'file %s, line %d: %s' % (filename, reader.line_num, e)
    
#     col_resize = list()
    
#     for i in range(rows):
#         tmp = list()
        
#         for j in range(len(col_list)):
#             tmp.append(col_list[j][i])
            
#         col_resize.append(tmp)
    
#     return col_resize


# get well xy pos for num
# for i in range(len(well_map)):
#     for j in well_map[i]:
#         print j
#         well_pos = j-1
#         well_col = abs(well_pos/rows)+1
#         well_row = abs(well_pos%rows)
        
#         print well_col
#         print well_row

In [104]:
import csv, sys
import numpy as np
import pandas as pd


In [90]:
def read_plate_file_to_csv(filename):
    
    csv_list = list()
    
    def get_csv_from_row(row):
        return row.split('=')[1].split(' ')
    
    with open(filename) as f:
        content = f.readlines()
    
    for line in content:
        # -1 is not found
        if line.find('Row') > 0:
            csv_list.append(get_csv_from_row(line.strip()))
            
    return csv_list

In [166]:
# from csv to plate array
# def read_plate_csv(filename):
#     import csv, sys
    
#     csv_list = list()
#     with open(filename, 'rb') as f:
#         reader = csv.reader(f)
#         try:
#             for row in reader:
#                 csv_list.append(row)
#         except csv.Error as e:
#             # TODO handle
#             print 'file %s, line %d: %s' % (filename, reader.line_num, e)
#     print 'oo'
#     print csv_list
#     return csv_list

# from plate array to mapped plate array
def get_mapped_plate(plate, filename='elisawells.csv',
                 rows=8, cols=12):
    import csv, sys
    
    row_count = 0
    col_count = 0
    
    mapped_plate = list()
       
    with open(filename, 'rb') as f:
        reader = csv.reader(f)
        col_count_list = list()

        for row in reader:
            row_value = int(row[0])
            ab2_value = float(row[1])
    
            well_pos = row_value
            well_col = abs(well_pos/rows)
            well_row = abs(well_pos%rows)-1
            
            mapped_plate.append((row_value, ab2_value, plate[well_row][well_col]))
            
    return mapped_plate


In [168]:
def get_prot_concentration_step(p_high):
    p = p_high

    return [0, p/8.0, p/4.0, p/2.0, p]

def get_coating_ab_step(a_high):
    p = a_high

    return [p/8.0, p/4.0, p/2.0, p]

def get_well_attrs(mapped_plate, prot_concentration_high, coating_ab, ab2_step):
    prot_step = get_prot_concentration_step(prot_concentration_high)
    coating_step = get_coating_ab_step(coating_ab)

    count = 0
    well_attr = list()
    for i in prot_step:
        for j in coating_step:
            for k in range(ab2_step):
                # join both tuples
                tmp_tup = ((j,i)+(mapped_plate[count]))
                well_attr.append({'coating_ab': tmp_tup[0],
                                  'prot': tmp_tup[1],
                                  'well': tmp_tup[2],
                                  'ab2': tmp_tup[3],
                                  'value': float(tmp_tup[4]),
                                 })
                count = count + 1
    return well_attr

In [169]:
# plate = read_plate_csv('plate_1_dummy.csv') # old


In [170]:
prot_concentration_high = 2.0
coating_ab = 4.0
rows = 8
cols = 12

# ab2_step = 4
# how many ab2 steps determines coating #

# plate from file to pandas dataframe for bokeh app
def read_plate_to_df(plate_file_csv,
                    prot_concentration_high=prot_concentration_high,
                    coating_ab=coating_ab,
                    ab2_step=4):

    plate = read_plate_file_to_csv(plate_file_csv)
    mapped_plate = get_mapped_plate(plate)
    well_attr = get_well_attrs(mapped_plate,
                               prot_concentration_high,
                               coating_ab,
                               ab2_step)
    df = pd.DataFrame(well_attr)
    return df

plate_file = '161102-001.CSV'
df = read_plate_to_df(plate_file,
                     prot_concentration_high,
                     coating_ab)

In [171]:
# plot one series
#df[(df.prot == 2) & (df.coating_ab == 1)]
#df

In [172]:
#prot_step = get_prot_concentration_step(prot_concentration_high)
#    for p in prot_step:

# get plot's worth of data points
def get_df_by_prot_conc(df, prot_conc):
    return df[(df.prot == prot_conc)]

print df.prot.unique()

prot_conc_05 = get_df_by_prot_conc(df, 0.5)
prot_conc_05.ab2.unique()

# get each point (coating) data 
#get_scatter_df_by_coating
#df['value'] = df['value'].astype('float64') 

df.dtypes
#df.value

[ 0.    0.25  0.5   1.    2.  ]


ab2           float64
coating_ab    float64
prot          float64
value         float64
well            int64
dtype: object

In [184]:
ab2_steps = prot_conc_05.sort_values('ab2').ab2.unique().astype(str).tolist()

In [185]:
ab2_steps

['0.25', '0.5', '1.0', '2.0']

In [ ]:
['0.25', '0.5', '1.0', '2.0']